In [10]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """
    Walk upwards from `start` (or CWD) until we find a repo marker.
    Works in notebooks where __file__ is not available.
    """
    start = (start or Path.cwd()).resolve()
    markers = ("pyproject.toml", ".git", "setup.cfg", "requirements.txt")
    for p in (start, *start.parents):
        if any((p / m).exists() for m in markers):
            return p
    # last resort: assume current folder is repo root
    return start

REPO_ROOT = find_repo_root()
OUT_DIR = REPO_ROOT / "data" / "signals"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("CWD:", Path.cwd())
print("REPO_ROOT:", REPO_ROOT)
print("OUT_DIR:", OUT_DIR)

CWD: r:\scanner\books
REPO_ROOT: C:\Users\MaartenEnde\Repos\scanner
OUT_DIR: C:\Users\MaartenEnde\Repos\scanner\data\signals


In [11]:
from pathlib import Path
import pandas as pd
import numpy as np

from scanner_bot.infra.db.sqllite_barstore import SQLiteBarStore

DB_PATH = (REPO_ROOT / "data" / "db" / "scanner.db").resolve()

store = SQLiteBarStore(DB_PATH)

# Option A: all tickers in DB (fine if DB isn't huge)
bars = store.list_bars_multi(interval="1d", tickers=None)

# Option B: restrict to a list (uncomment)
# tickers = ["AAPL", "MSFT", "NVDA"]
# bars = store.list_bars_multi(interval="1d", tickers=tickers)

def _px(b):
    # prefer adj_close when present
    return b.adj_close if getattr(b, "adj_close", None) is not None else b.close

df = pd.DataFrame(
    {
        "date": [b.date for b in bars],
        "ticker": [b.ticker for b in bars],
        "px": [_px(b) for b in bars],
    }
).dropna(subset=["px"])

df["date"] = pd.to_datetime(df["date"])
df.head(), df["ticker"].nunique(), df["date"].min(), df["date"].max()





(        date ticker        px
 0 1972-06-01    PEP  0.377921
 1 1972-06-01    TXN  0.695581
 2 1972-06-02    PEP  0.376828
 3 1972-06-02    TXN  0.708354
 4 1972-06-05    PEP  0.372447,
 35,
 Timestamp('1972-06-01 00:00:00'),
 Timestamp('2026-01-06 00:00:00'))

In [12]:
px_m = (
    df.set_index("date")
      .groupby("ticker")["px"]
      .resample("M")
      .last()
      .unstack("ticker")
      .sort_index()
)

print("Months:", len(px_m), "| Tickers:", px_m.shape[1])
px_m.tail()


C:\Users\MaartenEnde\AppData\Local\Temp\ipykernel_4344\1805412299.py:4: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample("M")


Months: 644 | Tickers: 35


ticker,AAPL,ADBE,ADP,AMAT,AMD,AMGN,AMZN,ASML,AVGO,BIDU,...,NDAQ,NFLX,NVDA,PEP,PLTR,PYPL,QCOM,SBUX,TMUS,TXN
date,,,,,,,,,,,,,,,,,,,,,
2025-09-30,254.383408,352.750000,291.614960,204.339462,161.789993,280.201508,219.570007,966.375488,329.279968,131.770004,...,88.185501,119.891998,186.569611,139.079666,182.419998,66.905327,165.514282,83.993195,238.199890,182.104568
2025-10-31,270.108154,340.309998,258.628204,232.643982,256.119995,296.316559,244.220001,1059.229980,368.924103,120.870003,...,85.234352,111.886002,202.478729,144.674942,200.470001,69.110229,179.980362,80.289955,209.014481,161.460007
2025-11-30,278.850006,320.130005,253.660324,252.250000,217.529999,345.459991,233.220001,1060.000000,402.190460,116.889999,...,90.648117,107.580002,176.990143,147.299271,168.449997,62.689999,167.235474,87.110001,209.009995,168.270004
2025-12-31,271.859985,349.989990,257.230011,256.989990,214.160004,327.309998,230.820007,1069.859985,346.100006,130.660004,...,97.129997,93.760002,186.500000,143.520004,177.750000,58.380001,171.050003,84.209999,203.039993,173.490005
2026-01-31,262.359985,335.989990,261.119995,296.010010,214.350006,330.170013,240.929993,1242.189941,343.769989,146.419998,...,100.690002,90.650002,187.240005,138.960007,179.710007,59.810001,182.449997,89.459999,198.600006,192.100006


In [15]:
import pandas as pd
import numpy as np
from pathlib import Path

TOP_PCT = 0.30
BOT_PCT = 0.30
LOOKBACK_MONTHS = 12      # "last 12 months"
EXCLUDE_CURRENT = True    # use Pt-1 / Pt-13
MIN_TICKERS = 10          # skip months with too few names

# must already exist
assert "px_m" in globals(), "Expected px_m (month-end price matrix) to exist."
px_m = px_m.sort_index()

print("px_m shape:", px_m.shape)
print("date range:", px_m.index.min(), "->", px_m.index.max())
print("example tickers:", list(px_m.columns[:10]))


px_m shape: (644, 35)
date range: 1972-06-30 00:00:00 -> 2026-01-31 00:00:00
example tickers: ['AAPL', 'ADBE', 'ADP', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'ASML', 'AVGO', 'BIDU']


In [16]:
# log return for current month: log(Pt / Pt-1)
logret_1m = np.log(px_m / px_m.shift(1))

# 12M momentum excluding current period: log(Pt-1 / Pt-13)
# (Pt-1 is shift(1), Pt-13 is shift(13))
mom_12m_excl = np.log(px_m.shift(1) / px_m.shift(13))

# optional: for readability in excel
ret_1m_simple = np.exp(logret_1m) - 1
mom_12m_simple = np.exp(mom_12m_excl) - 1

logret_1m.head()


ticker,AAPL,ADBE,ADP,AMAT,AMD,AMGN,AMZN,ASML,AVGO,BIDU,...,NDAQ,NFLX,NVDA,PEP,PLTR,PYPL,QCOM,SBUX,TMUS,TXN
date,,,,,,,,,,,,,,,,,,,,,
1972-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1972-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.040881,NaN,NaN,NaN,NaN,NaN,0.018138
1972-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.012960,NaN,NaN,NaN,NaN,NaN,0.019475
1972-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,-0.030998,NaN,NaN,NaN,NaN,NaN,0.007118
1972-10-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.001498,NaN,NaN,NaN,NaN,NaN,0.008965


In [ ]:
def monthly_select_and_portfolio(
    px_m: pd.DataFrame,
    logret_1m: pd.DataFrame,
    mom_12m_excl: pd.DataFrame,
    top_pct: float = 0.30,
    bot_pct: float = 0.30,
    min_tickers: int = 10,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Returns:
      panel: rows = (date, ticker) with price, returns, momentum, selection
      port : rows = date with winners, losers (short contrib), wml, counts
    """
    panel_rows = []
    port_rows = []

    for dt in px_m.index:
        s = mom_12m_excl.loc[dt].dropna()
        r = logret_1m.loc[dt].dropna()
        p = px_m.loc[dt].dropna()

        common = s.index.intersection(r.index).intersection(p.index)
        n = len(common)
        if n < min_tickers:
            continue

        k = int(np.floor(n * top_pct))
        k2 = int(np.floor(n * bot_pct))
        if k <= 0 or k2 <= 0:
            continue

        s = s.loc[common]
        r = r.loc[common]
        p = p.loc[common]

        ranked = s.sort_values(ascending=False)
        winners = ranked.index[:k]
        losers  = ranked.index[-k2:]

        # selection vector: 1 / 0 / -1
        sel = pd.Series(0, index=common, dtype=int)
        sel.loc[winners] = 1
        sel.loc[losers] = -1

        # portfolio log returns (equal weight)
        winners_ret = float(r.loc[winners].mean())          # log return of winners (long)
        losers_long = float(r.loc[losers].mean())           # log return of losers if held long

        wml_spread = winners_ret - losers_long              # spread in log space (your current W-L)
        wml_cap05  = 2.0 * wml_spread                       # “return on 0.5 capital” (your convention)

        port_rows.append(
            {
                "date": dt,
                "n_tickers": n,
                "k_winners": k,
                "k_losers": k2,
                "winners_logret": winners_ret,
                "losers_long_logret": losers_long,
                "wml_spread_logret": wml_spread,
                "wml_cap05_logret": wml_cap05,
            }
        )

        # per-ticker panel rows for excel
        out = pd.DataFrame(
            {
                "date": dt,
                "ticker": common,
                "price": p.values,
                "logret_1m": r.values,
                "mom_12m_excl_log": s.values,
                "selected": sel.values,  # 1 / 0 / -1
            }
        )
        panel_rows.append(out)

    panel = pd.concat(panel_rows, ignore_index=True) if panel_rows else pd.DataFrame()
    port = pd.DataFrame(port_rows).set_index("date").sort_index() if port_rows else pd.DataFrame()
    return panel, port

panel, port = monthly_select_and_portfolio(
    px_m=px_m,
    logret_1m=logret_1m,
    mom_12m_excl=mom_12m_excl,
    top_pct=TOP_PCT,
    bot_pct=BOT_PCT,
    min_tickers=MIN_TICKERS,
)

print("panel rows:", len(panel), "| portfolio months:", len(port))
port.head()


panel rows: 12825 | portfolio months: 499


,n_tickers,k_winners,k_losers,winners_logret,losers_short_logret,wml_logret
date,,,,,,
1984-07-31,10,3,3,-0.033850,0.128793,0.094944
1984-08-31,10,3,3,0.096663,-0.133262,-0.036599
1984-09-30,10,3,3,-0.085121,0.075836,-0.009285
1984-10-31,10,3,3,-0.019476,0.136552,0.117076
1984-11-30,10,3,3,0.023212,0.000501,0.023713


In [19]:
if not panel.empty:
    panel["ret_1m"] = np.exp(panel["logret_1m"]) - 1
    panel["mom_12m_excl"] = np.exp(panel["mom_12m_excl_log"]) - 1

# Sanity checks per month: sum(selected==1)=k_winners and sum(selected==-1)=k_losers
if not panel.empty and not port.empty:
    sel_counts = (
        panel.groupby("date")["selected"]
             .agg(winners=lambda x: int((x == 1).sum()),
                  losers=lambda x: int((x == -1).sum()))
    )
    chk = port.join(sel_counts, how="left")
    chk["ok_winners"] = chk["winners"] == chk["k_winners"]
    chk["ok_losers"] = chk["losers"] == chk["k_losers"]
    print(chk[["k_winners","winners","ok_winners","k_losers","losers","ok_losers"]].tail(12))

# Portfolio simple returns too (optional)
if not port.empty:
    port["winners_ret"] = np.exp(port["winners_logret"]) - 1
    port["losers_short_ret"] = np.exp(port["losers_short_logret"]) - 1
    port["wml_ret"] = np.exp(port["wml_logret"]) - 1

port.tail()


            k_winners  winners  ok_winners  k_losers  losers  ok_losers
date                                                                   
2025-02-28         10       10        True        10      10       True
2025-03-31         10       10        True        10      10       True
2025-04-30         10       10        True        10      10       True
2025-05-31         10       10        True        10      10       True
2025-06-30         10       10        True        10      10       True
2025-07-31         10       10        True        10      10       True
2025-08-31         10       10        True        10      10       True
2025-09-30         10       10        True        10      10       True
2025-10-31         10       10        True        10      10       True
2025-11-30         10       10        True        10      10       True
2025-12-31         10       10        True        10      10       True
2026-01-31         10       10        True        10      10    

,n_tickers,k_winners,k_losers,winners_logret,losers_short_logret,wml_logret,winners_ret,losers_short_ret,wml_ret
date,,,,,,,,,
2025-09-30,35,10,10,0.030915,-0.039974,-0.009059,0.031398,-0.039185,-0.009018
2025-10-31,35,10,10,0.085718,0.087248,0.172965,0.089499,0.091167,0.188825
2025-11-30,35,10,10,-0.017554,0.012952,-0.004602,-0.017401,0.013036,-0.004592
2025-12-31,35,10,10,0.008068,-0.018576,-0.010508,0.008100,-0.018405,-0.010453
2026-01-31,35,10,10,0.068464,-0.005758,0.062706,0.070862,-0.005741,0.064714


In [21]:
# --- Wide per-ticker block (MultiIndex columns: (ticker, metric)) ---
price_w   = panel.pivot(index="date", columns="ticker", values="price")
lr1_w     = panel.pivot(index="date", columns="ticker", values="logret_1m")
lr12_w    = panel.pivot(index="date", columns="ticker", values="mom_12m_excl_log")  # your Pt-1/Pt-13 log
sel_w     = panel.pivot(index="date", columns="ticker", values="selected")

# Build as (metric, ticker) then swap to (ticker, metric)
tickers_block = pd.concat(
    {
        "price": price_w,
        "logret_1m": lr1_w,
        "logret_12m": lr12_w,
        "selected": sel_w,
    },
    axis=1
)
tickers_block = tickers_block.swaplevel(0, 1, axis=1).sort_index(axis=1, level=0)

# --- Portfolio block: (PORTFOLIO, metric) ---
port_cols = pd.DataFrame(
    {
        "winner": port["winners_logret"],
        "loser": port["losers_short_logret"],
        "winner_minus_loser": port["wml_logret"],
    },
    index=port.index
)
port_cols.columns = pd.MultiIndex.from_product([["PORTFOLIO"], port_cols.columns])

# --- Combine + add DATE as first column with a 2-row header ---
out = tickers_block.join(port_cols, how="inner").sort_index()

out = out.reset_index()  # date becomes a column
out.columns = pd.MultiIndex.from_tuples([("DATE", "")] + list(out.columns[1:]))

out.head()



DATE       AAPL                                    ADBE            \
             logret_12m logret_1m     price selected logret_12m logret_1m   
0 1984-07-31  -0.612118 -0.038473  0.087267     -1.0        NaN       NaN   
1 1984-08-31  -0.313095  0.038473  0.090690     -1.0        NaN       NaN   
2 1984-09-30  -0.340505 -0.053286  0.085984     -1.0        NaN       NaN   
3 1984-10-31   0.082944 -0.010000  0.085128      0.0        NaN       NaN   
4 1984-11-30   0.094811 -0.005037  0.084701      1.0        NaN       NaN   

                        ADP  ...      TMUS                       TXN  \
  price selected logret_12m  ... logret_1m price selected logret_12m   
0   NaN      NaN  -0.157578  ...       NaN   NaN      NaN   0.093335   
1   NaN      NaN  -0.098888  ...       NaN   NaN      NaN   0.095853   
2   NaN      NaN   0.093843  ...       NaN   NaN      NaN   0.232306   
3   NaN      NaN   0.090084  ...       NaN   NaN      NaN   0.105669   
4   NaN      NaN   0.082616  ...       NaN   NaN      NaN   0.022193   

                               PORTFOLIO                               
  logret_1m     price selected    winner     loser winner_minus_loser  
0 -0.084016  1.219228      1.0 -0.033850  0.128793           0.094944  
1  0.198557  1.487022      1.0  0.096663 -0.133262          -0.036599  
2 -0.146103  1.284888      1.0 -0.085121  0.075836          -0.009285  
3  0.009027  1.296539      0.0 -0.019476  0.136552           0.117076  
4 -0.059566  1.221565      0.0  0.023212  0.000501           0.023713  

[5 rows x 144 columns]

In [23]:
analysis_dir = (REPO_ROOT / "data" / "analysis")
analysis_dir.mkdir(parents=True, exist_ok=True)
out_path = analysis_dir / "momentum_30_30_monthly_wide.xlsx"

# pandas can't write MultiIndex columns with index=False -> so use DATE as index
if ("DATE", "") in out.columns:
    out_xl = out.set_index(("DATE", ""))
else:
    out_xl = out.copy()

out_xl.index.name = "DATE"

with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    out_xl.to_excel(writer, sheet_name="monthly", index=True)  # <-- key change

    ws = writer.sheets["monthly"]
    ws.freeze_panes = "B3"  # freeze after DATE column + 2 header rows

print("Saved:", out_path)


Saved: C:\Users\MaartenEnde\Repos\scanner\data\analysis\momentum_30_30_monthly_wide.xlsx


In [24]:
import numpy as np
import pandas as pd

def _max_dd_from_logrets(r_log: pd.Series) -> float:
    r_log = r_log.dropna()
    if r_log.empty:
        return np.nan
    equity = np.exp(r_log.cumsum())          # log-additive -> equity curve
    dd = equity / equity.cummax() - 1.0
    return float(dd.min())

def _stats_from_logrets(r_log: pd.Series) -> dict:
    r_log = r_log.dropna()
    if r_log.empty:
        return {
            "months": 0, "total_log": np.nan, "total_return": np.nan,
            "cagr": np.nan, "vol": np.nan, "sharpe": np.nan, "maxdd": np.nan,
        }

    months = len(r_log)
    total_log = float(r_log.sum())
    total_return = float(np.exp(total_log) - 1.0)

    years = months / 12.0
    cagr = float(np.exp(total_log / years) - 1.0) if years > 0 else np.nan

    vol = float(r_log.std(ddof=1) * np.sqrt(12)) if months > 1 else np.nan
    sharpe = float((r_log.mean() * 12) / (r_log.std(ddof=1) * np.sqrt(12))) if months > 1 else np.nan

    maxdd = _max_dd_from_logrets(r_log)

    return {
        "months": months,
        "total_log": total_log,
        "total_return": total_return,
        "cagr": cagr,
        "vol": vol,
        "sharpe": sharpe,
        "maxdd": maxdd,
    }

def decade_year(dt: pd.Timestamp) -> int:
    return (dt.year // 10) * 10  # decade starts at YYYY-01-01 where YYYY ends with 0

# ensure datetime index
port_idx = pd.to_datetime(port.index)
port2 = port.copy()
port2.index = port_idx

cols = ["winners_logret", "losers_short_logret", "wml_logret"]
dec_rows = []

for dy, g in port2[cols].groupby(port2.index.map(decade_year)):
    start = g.index.min().date()
    end = g.index.max().date()

    row = {"decade_start_year": dy, "start": start, "end": end}
    for c in cols:
        s = _stats_from_logrets(g[c])
        prefix = c.replace("_logret", "")  # winners / losers_short / wml
        row.update({f"{prefix}_{k}": v for k, v in s.items()})
    dec_rows.append(row)

decades = pd.DataFrame(dec_rows).sort_values("decade_start_year").set_index("decade_start_year")
decades


,start,end,winners_months,winners_total_log,winners_total_return,winners_cagr,winners_vol,winners_sharpe,winners_maxdd,losers_short_months,...,losers_short_vol,losers_short_sharpe,losers_short_maxdd,wml_months,wml_total_log,wml_total_return,wml_cagr,wml_vol,wml_sharpe,wml_maxdd
decade_start_year,,,,,,,,,,,,,,,,,,,,,
1980,1984-07-31,1989-12-31,66,1.422576,3.147791,0.295181,0.368831,0.701270,-0.425241,66,...,0.430176,-0.057699,-0.571602,66,1.286062,2.618509,0.263429,0.339879,0.687978,-0.316013
1990,1990-01-31,1999-12-31,120,4.766187,116.470506,0.610619,0.310911,1.532976,-0.244146,120,...,0.313866,-0.807690,-0.936071,120,2.231121,8.310300,0.249961,0.292078,0.763878,-0.324581
2000,2000-01-31,2009-12-31,120,0.453649,0.574045,0.046410,0.285722,0.158773,-0.575273,120,...,0.368916,-0.190873,-0.784930,120,-0.250513,-0.221599,-0.024740,0.319916,-0.078306,-0.540136
2010,2010-01-31,2019-12-31,120,2.449157,10.578577,0.277514,0.184679,1.326171,-0.228047,120,...,0.186172,-0.872175,-0.818654,120,0.825411,1.282820,0.086043,0.154727,0.533462,-0.211142
2020,2020-01-31,2026-01-31,73,1.036974,1.820667,0.185852,0.215877,0.789621,-0.359882,73,...,0.214191,-0.376131,-0.480236,73,0.546877,0.727848,0.094062,0.204128,0.440397,-0.373291
